# Movie Trends Analysis

#### by Nadir Sarigul

## IV. Data Preparation

After careful exploration of all datasets made available to me by flatiron, I decided to focus my analyis on data extracted from 4 of the different datasets To prepare the dataframe necessary for this analysis I took a few different steps aimed at cleaning and organizing  and integrating the relevant data:
 
[1) Datasets imported:](#1) 
            
[2) Cleaning and preparation of financial-related data:](#2)
        

[3) Cleaning and preparation of ratings-related data:](#3)
             
[4) Creation of main dataset for modeling and data visualization:](#4)
            

In [1]:
import os,glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
folder = "zippedData/"
os.listdir(folder)
files = glob.glob(f"{folder}*.csv*")


### <a id='a'>Datasets imported </a>
     "tn_movie_budget", "imdb_title_basic", "tdmb_movies"

In [2]:
tn_movie_budgets =pd.read_csv("zippedData/tn.movie_budgets.csv.gz")
tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [3]:
imdb_title_basic = pd.read_csv("zippedData/imdb.title.basics.csv.gz")
imdb_title_basic.head(2)


,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


In [4]:
#drop Unnamed: 0 column

In [5]:
tdmb_movies = pd.read_csv("zippedData/tmdb.movies.csv.gz")
tdmb_movies.drop("Unnamed: 0", axis =1, inplace =True)
tdmb_movies.head(2)

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610


### <a id = "2">Cleaning and preparation</a>

 The finantial information utilized in this analysis was extracted from "tn_movie_budget" dataset
    
* In this dataset I first started by making sure that all the data available here was in the right format to enable subsequent analysis. To achieve this goal, I transformed the data type for the variables "production budget", "domestic gross" and "worldwide gross" from strings (the original format of the data) to intergers.


* To facilitate the data analysis I created a new table which I named "budgets" that contains only the variables of relevance for this analysis ("movie", "production_budget","domestic_gross", "worldwide_gross") by using the copying function from the original "tn_movie_budjet" 


In [6]:
#changing production_budget, domestic_gross, worldwide_gross to the right data type

In [7]:

#strip character 
tn_movie_budgets["production_budget"]= tn_movie_budgets["production_budget"].str.strip("$")
#remove comma using replace function
tn_movie_budgets["production_budget"]= tn_movie_budgets["production_budget"].str.replace(",","")
#turning data type to intiger using astype function
tn_movie_budgets["production_budget"] =tn_movie_budgets["production_budget"].astype(int)




In [8]:
tn_movie_budgets["domestic_gross"]= tn_movie_budgets["domestic_gross"].str.strip("$")
tn_movie_budgets["domestic_gross"]= tn_movie_budgets["domestic_gross"].str.replace(",","")
tn_movie_budgets["domestic_gross"] = tn_movie_budgets["domestic_gross"].astype(int)




In [9]:
tn_movie_budgets["worldwide_gross"]= tn_movie_budgets["worldwide_gross"].str.strip("$")
tn_movie_budgets["worldwide_gross"]= tn_movie_budgets["worldwide_gross"].str.replace(",","")
tn_movie_budgets["worldwide_gross"] = tn_movie_budgets["worldwide_gross"].astype(int)




* To have a direct metric of the profit made in each genre, I calculated the profit for each entry in this data set by subtracting the "budget" from "worldwide gross" and removed the entries with negative profit values from the dataset. This metric was added to the "budgets" table.

In [10]:
# copy tn_movie_budgets table with columns needed
budgets = tn_movie_budgets[["movie", "production_budget","domestic_gross","worldwide_gross"]].copy()


* I also calculated the profit margin for each entry in this dataset, as this metric allows for a better understanding of the profitability of each genre. To avoid the creation of non-values in the profit margin variable, I added a very small number (0.0001) in my formula to calculate the profit margin. The addition of this small value has no impact in the outcome of the analysis but avoids the creation of non-values which cause problems down the line with the analysis of the data. This metric was added to the "budgets" table. 


In [11]:
#add a profit column,  we can create a profitability ratio
budgets["profit"] = budgets["worldwide_gross"] - budgets["production_budget"]

#make sure no negative numbers for profit
budgets.loc[budgets["profit"] < 0, 'profit'] = 0
#create profit margin
budgets["profit_margin"] = (budgets["profit"]/(budgets["worldwide_gross"]+.0001))*100



In [12]:
budgets.describe()

,production_budget,domestic_gross,worldwide_gross,profit,profit_margin
count,5.782000e+03,5.782000e+03,5.782000e+03,5.782000e+03,5782.000000
mean,3.158776e+07,4.187333e+07,9.148746e+07,6.356163e+07,39.379167
std,4.181208e+07,6.824060e+07,1.747200e+08,1.441625e+08,35.787751
min,1.100000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,5.000000e+06,1.429534e+06,4.125415e+06,0.000000e+00,0.000000
50%,1.700000e+07,1.722594e+07,2.798445e+07,8.550286e+06,41.462600
75%,4.000000e+07,5.234866e+07,9.764584e+07,6.096850e+07,73.392547
max,4.250000e+08,9.366622e+08,2.776345e+09,2.351345e+09,99.944444


### <a id=3> Cleaning and preparation of ratings-related data:</a>

* The ratings information utilized in this analysis was extracted from "imdb_title_basic"and "tdmb_movies" datasets.


* To prepare the "imdb_title_basic" dataset for posterior utilization I removed the entries that did not have a genre associated with it 


In [13]:
#see total number of null  entries in genres column
imdb_title_basic["genres"].isna().sum()
imdb_title_basic[imdb_title_basic["genres"].isna()]
#drop null value
imdb_title_basic.dropna(subset =["genres"], inplace =True)

In [14]:
imdb_title_basic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140736 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           140736 non-null  object 
 1   primary_title    140736 non-null  object 
 2   original_title   140734 non-null  object 
 3   start_year       140736 non-null  int64  
 4   runtime_minutes  112233 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 7.5+ MB


* To create a more complete dataset that compiles information on ratings, year of release and genre for each movie entry, I merged the "tdmb_movies" dataset with the modified version of "imdb_title_basic" (described above) dataset. To achieve this I used the left merge operation having "original_title" as the matching key for the merge. This merged dataset was stored under the name "tdmb_imdb_title". 


In [15]:
# merge tdmb_movies to imdb_title_basic using "original title" collum as the matching key  and left merge operation
tdmb_imdb_title = pd.merge(tdmb_movies, imdb_title_basic, on='original_title', how="left")
tdmb_imdb_title.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31675 entries, 0 to 31674
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          31675 non-null  object 
 1   id                 31675 non-null  int64  
 2   original_language  31675 non-null  object 
 3   original_title     31675 non-null  object 
 4   popularity         31675 non-null  float64
 5   release_date       31675 non-null  object 
 6   title              31675 non-null  object 
 7   vote_average       31675 non-null  float64
 8   vote_count         31675 non-null  int64  
 9   tconst             20779 non-null  object 
 10  primary_title      20779 non-null  object 
 11  start_year         20779 non-null  float64
 12  runtime_minutes    19329 non-null  float64
 13  genres             20779 non-null  object 
dtypes: float64(4), int64(2), object(8)
memory usage: 3.6+ MB


In [16]:
tdmb_imdb_title

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,tconst,primary_title,start_year,runtime_minutes,genres
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,tt0926084,Harry Potter and the Deathly Hallows: Part 1,2010.0,146.0,"Adventure,Fantasy,Mystery"
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,tt0892769,How to Train Your Dragon,2010.0,98.0,"Action,Adventure,Animation"
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,tt1228705,Iron Man 2,2010.0,124.0,"Action,Adventure,Sci-Fi"
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,NaN,NaN,NaN,NaN,NaN
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,tt1375666,Inception,2010.0,148.0,"Action,Adventure,Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31670,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1,tt2962488,The Last One,2014.0,80.0,"Documentary,History,News"
31671,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1,tt3118968,The Last One,2018.0,85.0,"Action,Adventure,Fantasy"
31672,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1,tt6261158,The Last One,2017.0,NaN,Adventure
31673,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1,tt3377456,Trailer Made,2016.0,79.0,Adventure


In [17]:
#  runtime_minutes column to use for later

In [18]:
print(tdmb_imdb_title["runtime_minutes"].mean())
print(tdmb_imdb_title["runtime_minutes"].median())
print(tdmb_imdb_title["runtime_minutes"].mode())

91.51720213151224
90.0
0    90.0
dtype: float64


In [19]:
#replacing missing valeu with run time median 
tdmb_imdb_title["runtime_minutes"].fillna(tdmb_imdb_title["runtime_minutes"].median(),inplace= True)


In [20]:
tdmb_imdb_title

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,tconst,primary_title,start_year,runtime_minutes,genres
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,tt0926084,Harry Potter and the Deathly Hallows: Part 1,2010.0,146.0,"Adventure,Fantasy,Mystery"
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,tt0892769,How to Train Your Dragon,2010.0,98.0,"Action,Adventure,Animation"
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,tt1228705,Iron Man 2,2010.0,124.0,"Action,Adventure,Sci-Fi"
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,NaN,NaN,NaN,90.0,NaN
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,tt1375666,Inception,2010.0,148.0,"Action,Adventure,Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31670,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1,tt2962488,The Last One,2014.0,80.0,"Documentary,History,News"
31671,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1,tt3118968,The Last One,2018.0,85.0,"Action,Adventure,Fantasy"
31672,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1,tt6261158,The Last One,2017.0,90.0,Adventure
31673,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1,tt3377456,Trailer Made,2016.0,79.0,Adventure


* To facilitate the data analysis I created a new table which I named "tdmb_imdb" that contains only the variables of relevance for this analysis ("popularity", "release date","title", "vote_average", "vote_counts", "runtime_minutes", "genre") by using the copying function from the "tdmb_imdb_title" dataset.

In [21]:

tdmb_imdb = tdmb_imdb_title[["popularity","release_date","title","vote_average","vote_count","runtime_minutes","genres" ]].copy()

In [22]:
budgets

,movie,production_budget,domestic_gross,worldwide_gross,profit,profit_margin
0,Avatar,425000000,760507625,2776345279,2351345279,84.692106
1,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,60.733080
2,Dark Phoenix,350000000,42762350,149762350,0,0.000000
3,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,76.436443
4,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,75.925058
...,...,...,...,...,...,...
5777,Red 11,7000,0,0,0,0.000000
5778,Following,6000,48482,240495,234495,97.505146
5779,Return to the Land of Wonders,5000,1338,1338,0,0.000000
5780,A Plague So Pleasant,1400,0,0,0,0.000000


* The first step I took towards incorporating the finantial dataset with the ratings dataset was making sure that all variables in each dataset were ready to merge. To achieve this I changed the name of the colum "title" in the tdmb_imdb" datatset to "movie" to match the label of the "budgets" table.



* Once the datasets were appropriate for merging, I merged the "tdmb_imdb" with the"budgets" datasets using the inner join function and "movie" as matching key. This operation allows to get only entries for "movies" that have data in both tables. After merging, I further cleaned this new dataframe by dropping all null values and duplicates (again using "movies" as the directing entry).

In [23]:
# change the "title" column name to "movie" to have a common matching key 
#in order to be able to merge the budgets table with the tdmb_imdb table 

tdmb_imdb.rename(columns={"title": "movie"}, inplace=True)

# merge budgets and tdmb_imdb_title
budgets_tdmb_imdb =pd.merge(budgets,tdmb_imdb, on = "movie", how ="inner")
budgets_tdmb_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4100 entries, 0 to 4099
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie              4100 non-null   object 
 1   production_budget  4100 non-null   int64  
 2   domestic_gross     4100 non-null   int64  
 3   worldwide_gross    4100 non-null   int64  
 4   profit             4100 non-null   int64  
 5   profit_margin      4100 non-null   float64
 6   popularity         4100 non-null   float64
 7   release_date       4100 non-null   object 
 8   vote_average       4100 non-null   float64
 9   vote_count         4100 non-null   int64  
 10  runtime_minutes    4100 non-null   float64
 11  genres             3883 non-null   object 
dtypes: float64(4), int64(5), object(3)
memory usage: 416.4+ KB


In [24]:
# drop null values ( genres  217 null value)
#budgets_tdmb_imdb.isna().sum()

budgets_tdmb_imdb = budgets_tdmb_imdb.dropna()

# drop dublicated values
budgets_tdmb_imdb = budgets_tdmb_imdb.drop_duplicates(subset=["movie"])

### <a id = "4">Creation of main dataset for modeling and data visualization</a>

* Through the processes described above I created a new dataset which contains all the information that I need to answer the questions outlined above, which will be used as my dataframe for the data modeling and visualizations shown below. 

In [25]:
# save the dataframe i created to csv file to do visualization 

budgets_tdmb_imdb.to_csv (r'basic_data.csv', index = False, header=True)
df= pd.read_csv("basic_data.csv")
df

,movie,production_budget,domestic_gross,worldwide_gross,profit,profit_margin,popularity,release_date,vote_average,vote_count,runtime_minutes,genres
0,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,60.733080,30.579,2011-05-20,6.4,8571,136.0,"Action,Adventure,Fantasy"
1,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,76.436443,44.383,2015-05-01,7.3,13457,141.0,"Action,Adventure,Sci-Fi"
2,Avengers: Infinity War,300000000,678815482,2048134200,1748134200,85.352522,80.773,2018-04-27,8.3,13948,149.0,"Action,Adventure,Sci-Fi"
3,Justice League,300000000,229024295,655945209,355945209,54.264473,34.953,2017-11-17,6.2,7510,120.0,"Action,Adventure,Fantasy"
4,Spectre,300000000,200074175,879620923,579620923,65.894399,30.318,2015-11-06,6.4,6719,148.0,"Action,Adventure,Thriller"
...,...,...,...,...,...,...,...,...,...,...,...,...
1749,Exeter,25000,0,489792,464792,94.895792,5.934,2015-03-26,4.7,121,91.0,"Horror,Mystery,Thriller"
1750,Ten,25000,0,0,0,0.000000,1.575,2014-03-28,5.4,5,118.0,Drama
1751,Dry Spell,22000,0,0,0,0.000000,0.600,2013-02-14,6.0,1,90.0,"Comedy,Romance"
1752,All Superheroes Must Die,20000,0,0,0,0.000000,2.078,2013-01-04,3.9,19,78.0,"Sci-Fi,Thriller"
